# Practice: One step multivariate RNN model

In this notebook, we work through:
- preparing the time series data for training a RNN forecasting model
- get data in the required shape for the keras API
- implement a RNN model in keras to predict the next step ahead (time *t+1*) in the time series. This model uses recent values of temperature, as well as load, as the model input.
- enable early stopping to reduce the likelihood of model overfitting
- evaluate the model on a test dataset

The data in this example is taken from the GEFCom2014 forecasting competition<sup>1</sup>. It consists of 3 years of hourly electricity load and temperature values between 2012 and 2014. The task is to forecast future values of electricity load. In this example, we show how to forecast one time step ahead, using historical load and temperature data.

<sup>1</sup>Tao Hong, Pierre Pinson, Shu Fan, Hamidreza Zareipour, Alberto Troccoli and Rob J. Hyndman, "Probabilistic energy forecasting: Global Energy Forecasting Competition 2014 and beyond", International Journal of Forecasting, vol.32, no.3, pp 896-913, July-September, 2016.

In [ ]:
import matplotlib.pyplot as plt
%matplotlib inline

import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format

import numpy as np
np.set_printoptions(precision=2)

import warnings
warnings.filterwarnings("ignore")

import tensorflow as tf
tf.compat.v1.logging.set_verbosity(tf.compat.v1.logging.ERROR)

Load the data from `data/energy.parquet` into a Pandas dataframe

In [ ]:
import os

# Insert code START
file_name = None
energy = None
# Insert code END
assert energy.shape == (26304, 2)
energy.head()

## Create train, validation and test sets

We separate our dataset into train, validation and test sets. We train the model on the train set. The validation set is used to evaluate the model after each training epoch and ensure that the model is not overfitting the training data. After the model has finished training, we evaluate the model on the test set. We must ensure that the validation set and test set cover a later period in time from the training set, to ensure that the model does not gain from information from future time periods.

We will allocate the period 1st November 2014 to 31st December 2014 to the test set. The period 1st September 2014 to 31st October is allocated to validation set. All other time periods are available for the training set.

In [ ]:
# Insert code START
valid_start_dt = None
test_start_dt = None
# Insert code END

train = energy.copy()[:valid_start_dt]
valid = energy.copy()[valid_start_dt:test_start_dt]
test = energy.copy()[test_start_dt:]

assert train.index.max() == pd.to_datetime('2014-08-31 23:00:00')
assert valid.index.min() == pd.to_datetime('2014-09-01 00:00:00')
assert valid.index.max() == pd.to_datetime('2014-10-31 23:00:00')
assert test.index.min() == pd.to_datetime('2014-11-01 00:00:00')

## Data preparation

For this example, we will set *T=6*. This means that the input for each sample is a vector of the prevous 6 hours of the energy load. The choice of *T=6* was arbitrary but should be selected through experimentation.

*HORIZON=1* specifies that we have a forecasting horizon of 1 (*t+1*)

In [ ]:
T = 6
HORIZON = 1

### Data preparation - training set

In [ ]:
from sklearn.preprocessing import MinMaxScaler

# Fit a scaler for the y values
y_scaler = MinMaxScaler()
y_scaler.fit(train[['load']])

# Also scale the input features data (load and temp values)
X_scaler = MinMaxScaler()
train[['load', 'temp']] = X_scaler.fit_transform(train)
valid[['load', 'temp']] = X_scaler.transform(valid)
test[['load', 'temp']] = X_scaler.transform(test)

Use the TimeSeriesTensor convenience class to:
1. Shift the values of the time series to create a Pandas dataframe containing all the data for a single training example
2. Discard any samples with missing values
3. Transform this Pandas dataframe into a numpy array of shape (samples, time steps, features) for input into Keras

The class takes the following parameters:

- **dataset**: original time series
- **H**: the forecast horizon
- **tensor_structure**: a dictionary discribing the tensor structure in the form { 'tensor_name' : (range(max_backward_shift, max_forward_shift), [feature, feature, ...] ) }
- **freq**: time series frequency
- **drop_incomplete**: (Boolean) whether to drop incomplete samples

In [ ]:
from common.utils import TimeSeriesTensor

tensor_structure = {'X':(range(-T+1, 1), ['load', 'temp'])}
train_inputs = TimeSeriesTensor(dataset=train,
                            target='load',
                            H=HORIZON,
                            tensor_structure=tensor_structure,
                            freq='H',
                            drop_incomplete=True)


X_train = train_inputs['X']
y_train = train_inputs['target']

assert y_train.shape == (23370, 1)
assert X_train.shape == (23370, 6, 2)

#### Data preparation - validation set

In [ ]:
# In order to allow T lags, we need to add the last T-1 train samples to the validation set
valid = pd.concat([train.iloc[-(T-1):], valid])

# Create TimeSeriesTensor
valid_inputs = TimeSeriesTensor(valid, 'load', HORIZON, tensor_structure)
y_valid = valid_inputs['target']
X_valid = valid_inputs['X']

assert y_valid.shape == (1463, 1)
assert X_valid.shape == (1463, 6, 2)

## Quiz: Implement multivariate RNN

![Multivariate, multilayer RNN](./images/one_step_RNN_multivariate_mutilayer.png)

Implement your RNN model with the data prepared above and the following requirements:
1. Use 2 features: past load and temperature
2. Stack 2 GRU layers
3. 6 hidden units in the first GRU layer
4. 4 hidden units in the second GRU layer
5. 5 epochs
6. Batch size 32

In [ ]:
# Insert code START
FIRST_LAYER_LATENT_DIM = None  # number of units in the 1st RNN layer
SECOND_LAYER_LATENT_DIM = None # number of units in the 2nd RNN layer
BATCH_SIZE = None              # number of samples per mini-batch
EPOCHS = None                  # maximum number of times the training algorithm will cycle through all samples
# Insert code END

In [ ]:
from keras.models import Model, Sequential
from keras.layers import GRU, Dense
from keras.callbacks import EarlyStopping

#### Fill in your code below and replace the question mark

In [ ]:
model = Sequential()
# Fill in your code
# Hint: 3 model.add() statements.
#      There is a parameter you need to add when stacking multiple RNN layers

# Insert code START
# Insert code END

Once you done, run the rest of the notebook to check if your model works.

In [ ]:
model.compile(optimizer='RMSprop', loss='mse')
model.summary()

Specify the early stopping criteria. We **monitor** the validation loss (in this case the mean squared error) on the validation set after each training epoch. If the validation loss has not improved by **min_delta** after **patience** epochs, we stop the training.

In [ ]:
earlystop = EarlyStopping(monitor='val_loss', min_delta=0, patience=5)

history = model.fit(X_train,
                    y_train,
                    batch_size=BATCH_SIZE,
                    epochs=EPOCHS,
                    validation_data=(X_valid, y_valid),
                    callbacks=[earlystop],
                    verbose=1)

In [ ]:
plot_df = pd.DataFrame.from_dict({'train_loss':history.history['loss'], 'val_loss':history.history['val_loss']})
plot_df.plot(logy=True, figsize=(10,8), fontsize=12)
plt.xlabel('epoch', fontsize=12)
plt.ylabel('loss', fontsize=12)
plt.show()

## Evaluate the model

Create the test set

In [ ]:
# In order to allow T lags, we need to add the last T-1 validation samples to the test set
test = pd.concat([valid.iloc[-(T-1):], test])

# Create TimeSeriesTensor
test_inputs = TimeSeriesTensor(test, 'load', HORIZON, tensor_structure)
y_test = test_inputs['target']
X_test = test_inputs['X']

assert y_test.shape == (1463, 1)
assert X_test.shape == (1463, 6, 2)

In [ ]:
from common.utils import create_evaluation_df

predictions = model.predict(X_test)
eval_df = create_evaluation_df(predictions, test_inputs, HORIZON, y_scaler)
eval_df.head()

In [ ]:
from common.utils import mape

print('MAPE: {:.2f}%'.format(100 * mape(eval_df['prediction'], eval_df['actual'])))

In [ ]:
eval_df[eval_df.timestamp<'2014-11-08'].plot(x='timestamp', y=['prediction', 'actual'], style=['r', 'b'], figsize=(15, 8), fontsize=12)
plt.xlabel('timestamp', fontsize=12)
plt.ylabel('load', fontsize=12)
plt.show()